In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [2]:
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.precision',150)

In [3]:
df = pd.read_json('../data/whats-cooking/train.json')

In [4]:
def lower_list(arr):
    return [x.lower() for x in arr]

In [5]:
df['ingredient_count'] = df['ingredients'].map(lambda x: len(x))

In [6]:
df['ingredients'] = df['ingredients'].map(lower_list)

In [7]:
df.head(10)

,id,cuisine,ingredients,ingredient_count
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes...",9
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g...",11
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g...",12
3,22213,indian,"[water, vegetable oil, wheat, salt]",4
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe...",20
5,6602,jamaican,"[plain flour, sugar, butter, eggs, fresh ginge...",12
6,42779,spanish,"[olive oil, salt, medium shrimp, pepper, garli...",13
7,3735,italian,"[sugar, pistachio nuts, white almond bark, flo...",10
8,16903,mexican,"[olive oil, purple onion, fresh pineapple, por...",13
9,12734,italian,"[chopped tomatoes, fresh basil, garlic, extra-...",6


In [8]:
def remove_spaces(arr):
    return [x.replace(' ', '_') for x in arr]

In [9]:
def strip_words(arr):
     return [x.strip() for x in arr]

In [10]:
def remove_commas(arr):
    return [x.replace(',', '') for x in arr]

In [11]:
def remove_dots(arr):
    return [x.replace('.', '') for x in arr]

In [12]:
def remove_weird_characters(arr):
    weird_characters = [',', '.', '(', ')', "'"]
    for char in weird_characters:
        for word in arr:
            word.replace(char, '')
    return arr

In [13]:

df['ingredients'] = df['ingredients'].map(strip_words)
df['ingredients'] = df['ingredients'].map(remove_weird_characters)
df['ingredients'] = df['ingredients'].map(remove_spaces)


In [14]:
df['ingredients'].head(15)

0     [romaine_lettuce, black_olives, grape_tomatoes...
1     [plain_flour, ground_pepper, salt, tomatoes, g...
2     [eggs, pepper, salt, mayonaise, cooking_oil, g...
3                   [water, vegetable_oil, wheat, salt]
4     [black_pepper, shallots, cornflour, cayenne_pe...
5     [plain_flour, sugar, butter, eggs, fresh_ginge...
6     [olive_oil, salt, medium_shrimp, pepper, garli...
7     [sugar, pistachio_nuts, white_almond_bark, flo...
8     [olive_oil, purple_onion, fresh_pineapple, por...
9     [chopped_tomatoes, fresh_basil, garlic, extra-...
10    [pimentos, sweet_pepper, dried_oregano, olive_...
11    [low_sodium_soy_sauce, fresh_ginger, dry_musta...
12    [italian_parsley_leaves, walnuts, hot_red_pepp...
13    [ground_cinnamon, fresh_cilantro, chili_powder...
14    [fresh_parmesan_cheese, butter, all-purpose_fl...
Name: ingredients, dtype: object

In [15]:
df['ingredients'] = df['ingredients'].map(lambda x: " ".join(x)) 

In [16]:
df['ingredients']

0        romaine_lettuce black_olives grape_tomatoes ga...
1        plain_flour ground_pepper salt tomatoes ground...
2        eggs pepper salt mayonaise cooking_oil green_c...
3                           water vegetable_oil wheat salt
4        black_pepper shallots cornflour cayenne_pepper...
                               ...                        
39769    light_brown_sugar granulated_sugar butter warm...
39770    kraft_zesty_italian_dressing purple_onion broc...
39771    eggs citrus_fruit raisins sourdough_starter fl...
39772    boneless_chicken_skinless_thigh minced_garlic ...
39773    green_chile jalapeno_chilies onions ground_bla...
Name: ingredients, Length: 39774, dtype: object

In [17]:
pd.set_option('max_colwidth', 300)

In [26]:
print(df[df['ingredients'].str.contains('frozen_chopped_spinach')]['ingredients'])

177                                                                                        pasta_sauce fresh_parmesan_cheese olive_oil_flavored_cooking_spray frozen_chopped_spinach large_egg_whites part-skim_ricotta_cheese pepper large_eggs italian_seasoning manicotti_shells part-skim_mozzarella_cheese salt
311                                                                                    seasoned_bread_crumbs grated_parmesan_cheese salt fresh_parsley chicken_broth minced_garlic half_&_half shredded_mozzarella_cheese frozen_chopped_spinach pepper flour penne_pasta black_pepper large_eggs butter ground_beef
638                                                                                     tomato_sauce part-skim_mozzarella_cheese part-skim_ricotta_cheese frozen_chopped_spinach low-fat_cottage_cheese whole_wheat_lasagna_noodles shredded_parmesan_cheese eggs eggplant garlic olive_oil ground_black_pepper salt
680                                                                      

In [19]:
print(df[df['ingredients'].str.contains(',')]['ingredients'])

100                                                                                                                       tomato_sauce shredded_carrots spinach part-skim_mozzarella_cheese italian_seasoning english_muffins,_split_and_toasted chopped_onion vegetable_oil_cooking_spray chopped_green_bell_pepper
164                                                                                                 tomatoes olive_oil chopped_fresh_thyme salt chicken_bones green_bell_pepper,_slice black_olives lemon_juice eggs ground_black_pepper garlic anchovy_fillets dried_thyme lettuce_leaves purple_onion crusty_rolls
196                                                                                                                                                                                                       chopped_celery fire_roasted_diced_tomatoes andouille_sausage sweet_pepper red_kidnei_beans,_rins_and_drain
201                                                                      

In [20]:
count = CountVectorizer(min_df=2)

In [21]:
X = df['ingredients']
y = df['cuisine']

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [23]:
X_train_cv = count.fit_transform(X_train)
X_test_cv = count.transform(X_test)
df_X_train = pd.DataFrame(X_train_cv.todense(), columns=count.get_feature_names())

In [24]:
df_X_train['kosher_salt']

0        0
1        0
2        0
3        0
4        0
        ..
29825    0
29826    0
29827    0
29828    0
29829    1
Name: kosher_salt, Length: 29830, dtype: int64

In [25]:
df_X_train.head()

,10_oz,14,14_oz,15_oz,25,2_1,2_to_3_lb,33,40,5_oz,7_up,95,____oz,___oz,_alfredo_sauce,_all_purpose_sticks,_butter,_chicken,_chicken_broth,_chopped_green_chiles,_chunky_mild_salsa,_classico_olive_oil,_cook_and_drain,_cooked_and_drained,_cookies,_cream,_crush,_cut_into,_cut_into_italian_loaf,_cut_into_serving_pieces,_dash,_diced_tomatoes,_dips_mix,_drain,_drained_and_chopped,_dressing,_dressing_,_dried_fish,_eggs,_fiesta_sides,_fine_chop,_frozen_chopped_spinach,_green_chilies,_half,_lean_ground_beef,_less_sodium_chicken_broth,_less_sodium_ham,_less_sodium_taco_seasoning,_less_sodium_taco_seasoning_mix,_low,_mexican_rice,_mild_italian_ground_sausage,_natural_vanilla_ice_cream,_onion_soup_mix,_or_best_food_light_mayonnais,_or_best_food_real_mayonnais,_original_ranch,_original_ranch_salad,_pasta_sides,_peel_and_devein,_pepper,_perrins_worcestershire_sauce,_real_mayonnaise,_recipe_secrets,_reduced,_refried_beans,_refrigerated_crescent_dinner_rolls,_rins_and_drain,_seasoning_mix,_skinless_chicken_breast,_slice,_sliced,_soften,_split_and_toasted,_spread,_sweetened_condensed_milk,_taco_seasoning_mix,_thaw,_thaw_and_drain,_thawed_and_squeezed_dry,_thawed_and_undiluted,_thick_,_tomato_paste,_tomato_sauce,_traditional_cut_shredded_4_cheese_mexican,_undrain,_water,_well_scrub,_well_scrubbed,_yams,abalone,abura_age,accent_seasoning,achiote,achiote_paste,achiote_powder,acini_di_pepe,ackee,acorn_squash,acting_baking_powder,active_dry_yeast,added_black_beans,added_diced_tomatoes,adobo,adobo_sauce,adobo_seasoning,adzuki_beans,agar,agave_nectar,aged_balsamic_vinegar,aged_cheddar_cheese,ahi,ahi_tuna_steaks,aioli,ajinomoto,ajwain,aka_miso,alaskan_king_crab_legs,albacore_tuna_in_water,alcohol,ale,aleppo_pepper,alfredo_sauce,all,all_beef_hot_dogs,all_potato_purpos,all_purpose_seasoning,all_purpose_unbleached_flour,allspice,allspice_berries,almond_butter,almond_extract,almond_flour,almond_liqueur,almond_meal,almond_milk,almond_paste,almonds,alphabet_pasta,amaranth,amaretti,amaretti_cookies,amaretto,amaretto_liqueur,amchur,american_cheese,american_cheese_slices,ampalaya,anaheim_chile,anasazi_beans,ancho,ancho_chile_pepper,ancho_chili_ground_pepper,ancho_powder,anchovies,anchovy_filets,anchovy_fillets,anchovy_paste,and_fat_free_half_half,andouille_chicken_sausage,andouille_sausage,andouille_sausage_links,angel_food_cake,angel_food_cake_mix,angel_hair,angostura_bitters,anise,anise_extract,anise_oil,anise_powder,anise_seed,anisette,anjou_pears,annatto,annatto_seeds,apple_brandy,apple_butter,apple_cider,apple_cider_vinegar,apple_jelly,apple_juice,apple_juice_concentrate,apple_pie_filling,apple_pie_spice,apple_sausage,apples,applesauce,applewood_smoked_bacon,apricot_halves,apricot_jam,apricot_nectar,apricot_preserves,apricots,arbol_chile,arborio_rice,armagnac,arrowroot,arrowroot_powder,artichok_heart_marin,artichoke_bottoms,artichoke_hearts,artichokes,arugula,asadero,asafetida,asafetida_powder,asafoetida,asafoetida_powder,asiago,asian_basil,asian_chile_paste,asian_chili_red_sauc,asian_chili_sauce,asian_eggplants,asian_fish_sauce,asian_noodles,asian_pear,asian_rice_noodles,asian_wheat_noodles,asparagus,asparagus_spears,asparagus_tips,aspic,assorted_fresh_vegetables,atar,atta,au_jus_gravy_mix,avocado,avocado_leaves,avocado_oil,açai,açai_powder,baby_artichokes,baby_arugula,baby_back_ribs,baby_bok_choy,baby_broccoli,baby_carrots,baby_corn,baby_eggplants,baby_gem_lettuce,baby_greens,baby_leaf_lettuce,baby_lima_beans,baby_portobello_mushrooms,baby_potatoes,baby_radishes,baby_spinach,baby_spinach_leaves,baby_tatsoi,...,veal_demi,veal_for_stew,veal_loin_chops,veal_rib_chops,veal_scallops,veal_shanks,veal_shoulder,veal_stock,vegan_butter,vegan_margarine,vegan_mayonnaise,vegan_parmesan_cheese,vegan_sour_cream,vegan_worcestershire_sauce,vegetable_bouillon,vegetable_bouillon_cube,vegetable_broth,vegetable_demi,vegetable_gumbo,vegetable_juice,vegetable_juice_cocktail,vegetable_oil,vegetable_oil_cooking_spray,vegetable_oil_spray,vegetable_seasoning,vegetable_shortening,